In [ ]:
import os
import cv2 
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from skimage import transform
from dataset_loader import DatasetLoader
from annotations_loader import AnnotationsLoader

def display_img(img, BGR=False):
    if isinstance(img, str):
        display_img(cv2.imread(img))
        return
    plt.figure(dpi=150)
    if BGR:
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    else:
        plt.imshow(np.array(img))

In [ ]:
home_dir = '/proj/brizk/output/'
faces_dir = os.path.join(*[home_dir, 'retinaface'])
attention_dir = os.path.join(*[home_dir, 'attentiontarget'])
videos_loader = DatasetLoader()

In [ ]:
while(True):
    video = next(videos_loader)
    video_name = os.path.basename(video.filepath).split('.')[0]
    ds = videos_loader.current_ds
    print(f'Processing {video_name} @ {ds}')
    try:
        annotations = AnnotationsLoader(
            ds, video_name, faces_dir=faces_dir, attention_dir=attention_dir,
            faces_confidence_thres = 0.95
        )
        break
    except FileNotFoundError:
        print(f'Annotations not available yet')

### Looping over frames annotations (Targeting only at least of 2 attentions)

In [ ]:
annotations = AnnotationsLoader(
            ds, video_name, faces_dir=faces_dir, attention_dir=attention_dir,
            faces_confidence_thres = 0.95
        )

In [ ]:
passed = 0
faces_num_thres = 2
num_of_faces = 0

while(True):
    annotation = next(annotations)
    if annotation is None:
        break
    num_of_faces = len(annotation['faces'])
    if num_of_faces > faces_num_thres:
        break
    passed += 1

if num_of_faces > faces_num_thres:
    print('Discarded', passed, 'frames')
    frame_num = annotation['frame']
    print('Now at frame', frame_num)
else:
    print(f'Did not found any simulatanious more than {faces_num_thres} faces in video')

In [ ]:
annotation['faces']

In [ ]:
def calc_face_center(head_box):
    return [
        head_box[0] + (head_box[2] - head_box[0])//2,
        head_box[1] + (head_box[3] - head_box[1])//2
    ]

def get_attention_vector(head_box, observation_coordinates):
    face_center = calc_face_center(head_box)
    return np.append(face_center, observation_coordinates)

def calc_line_angle(line):
    return np.degrees(np.arctan2(-(line[3]-line[1]), line[2]-line[0]))

def calc_relative_angle(vectors, verbose=False):
    angles = np.array([calc_line_angle(l) for l in vectors])
    if verbose:
        print('Angles:', angles)
    return abs(angles[0] - angles[1])

In [ ]:
frame_img = cv2.cvtColor(video[frame_num], cv2.COLOR_BGR2RGB)
print('queried frame_num', frame_num)
print('current frame', video.current_frame_num)
height, width, _ = frame_img.shape
faces_per_frame = annotation['faces']
attention_per_frame = annotation['attention']
annotation['faces']

display_img(frame_img)

colors = [(255, 0, 0), (0, 0, 255), (0, 255, 0)]
attention_vectors = np.zeros((num_of_faces,4), dtype=np.int32)

for i in faces_per_frame.index:
    head_box = np.array(
        [faces_per_frame.loc[i,'left'], faces_per_frame.loc[i,'top'],
         faces_per_frame.loc[i,'right'], faces_per_frame.loc[i,'bottom']]
    ).astype(np.int32)
    
    observation_coordinates = attention_per_frame.loc[i, 'observation_coordinates']
    print('Observation is at', observation_coordinates)

    raw_hm = attention_per_frame.loc[i, 'raw_hm']
    inout = attention_per_frame.loc[i, 'inout']
    norm_map = transform.resize(raw_hm, (height, width)) - inout


    vector_of_attention = get_attention_vector(head_box, observation_coordinates)
    print('vector of attention:', vector_of_attention)
    cv2.line(frame_img, vector_of_attention[:2], vector_of_attention[2:], colors[i], 4)
    attention_vectors[i] = vector_of_attention
    
    cv2.rectangle(frame_img, (head_box[0], head_box[1]), (head_box[2], head_box[3]), colors[i], 4)
    cv2.circle(frame_img, observation_coordinates, int(height/50.0), colors[i], 4)

    ax = plt.gca()
    rect = patches.Rectangle((head_box[0], head_box[1]), head_box[2]-head_box[0], head_box[3]-head_box[1], linewidth=2, edgecolor=(0,1,0), facecolor='none')
    ax.add_patch(rect)

    circ = patches.Circle((observation_coordinates[0], observation_coordinates[1]), height/50.0, facecolor=(0,1,0), edgecolor='none')
    ax.add_patch(circ)
    
    plt.imshow(norm_map, cmap = 'jet', alpha=0.2, vmin=0, vmax=255)
    # Grayscale then Otsu's threshold
    gray = norm_map.copy()
    gray[gray < 0] = 0
    gray = gray.astype(np.uint8)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    # Find contours
    cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        # print('contour shape', c.shape)
        x,y,w,h = cv2.boundingRect(c)
        cv2.rectangle(frame_img, (x, y), (x + w, y + h), colors[i], 10)
        print('rect bounds', x, y, w, h)
        
    plt.plot((observation_coordinates[0],(head_box[0]+head_box[2])/2), (observation_coordinates[1],(head_box[1]+head_box[3])/2), '-', color=(0,1,0,1))
display_img(frame_img)

In [ ]:
# Testing two vectors looking opposite to each other
attention_vectors_dump = attention_vectors.copy()
attention_vectors_dump[1] = attention_vectors_dump[0].copy()
tmp = attention_vectors_dump[0][:2].copy()
attention_vectors_dump[0][:2] = attention_vectors_dump[0][2:].copy()
attention_vectors_dump[0][2:] = tmp
print(attention_vectors_dump)
calc_relative_angle(attention_vectors_dump, verbose=True)

In [ ]:
print(attention_vectors)
calc_relative_angle(attention_vectors, verbose=True)